In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# load ascii text 
filename = "my_words.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()

# OR Write your own:

# covert to lowercase
raw_text = raw_text.lower()

In [3]:
raw_text

"hollywood's bleeding, vampires feedin'\ndarkness turns to dust\neveryone's gone, but no one's leavin'\nnobody left but us\ntryna\u2005chase\u2005a feelin', but\u2005we'll never feel it\nridin' on the\u2005last train home\ndyin' in our sleep, we're living out a dream\nwe only make it out alone\ni just keep on hopin' that you call me\nyou say you wanna see me, but you can't right now\nyou never took the time to get to know me\nwas scared of losin' somethin' that we never found\nwe're running out of reasons, but we can't let go\nyeah, hollywood is bleeding, but we call it home\noutside, the winter sky turnin' grey\ncity up in smoke, it's only ash when it rains\nhowl at the moon and go to sleep in the day\nlove for everybody 'til the drugs fade away\nin the mornin', blocking out the sun with the shades\nshe gotta check her pulse and tell herself that she okay\nit seem like dying young is an honor\nbut who'd be at my funeral? i wonder\ni go out, and all they eyes on me\ni show out, do you 

In [4]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [5]:
char_to_int

{'\n': 0,
 ' ': 1,
 "'": 2,
 ',': 3,
 '-': 4,
 '?': 5,
 'a': 6,
 'b': 7,
 'c': 8,
 'd': 9,
 'e': 10,
 'f': 11,
 'g': 12,
 'h': 13,
 'i': 14,
 'j': 15,
 'k': 16,
 'l': 17,
 'm': 18,
 'n': 19,
 'o': 20,
 'p': 21,
 'r': 22,
 's': 23,
 't': 24,
 'u': 25,
 'v': 26,
 'w': 27,
 'y': 28,
 '\u2005': 29}

In [6]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  1485
Total Vocab:  30


In [7]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  1385


In [8]:
print(len(dataX))
print(len(dataX[0]))

1385
100


In [9]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [10]:
print(X.shape)
print(y.shape)

(1385, 100, 1)
(1385, 30)


In [11]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

W1112 09:10:12.672892 140021016172288 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1112 09:10:12.714646 140021016172288 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1112 09:10:12.719000 140021016172288 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1112 09:10:13.147665 140021016172288 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1112 09:10:13.160891 

In [14]:
# define the checkpoint
filepath="/home/debjoy/towards_vortex/beginnersNNS/RNN/models_seqgen/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [15]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1385/1385 [==============================] - 4s 3ms/step - loss: 2.9926

Epoch 00001: loss improved from inf to 2.99255, saving model to /home/debjoy/towards_vortex/beginnersNNS/RNN/models_seqgen/weights-improvement-01-2.9926.hdf5
Epoch 2/20
1385/1385 [==============================] - 4s 3ms/step - loss: 2.9729

Epoch 00002: loss improved from 2.99255 to 2.97285, saving model to /home/debjoy/towards_vortex/beginnersNNS/RNN/models_seqgen/weights-improvement-02-2.9729.hdf5
Epoch 3/20
1385/1385 [==============================] - 4s 3ms/step - loss: 2.9659

Epoch 00003: loss improved from 2.97285 to 2.96586, saving model to /home/debjoy/towards_vortex/beginnersNNS/RNN/models_seqgen/weights-improvement-03-2.9659.hdf5
Epoch 4/20
1385/1385 [==============================] - 4s 3ms/step - loss: 2.9653

Epoch 00004: loss improved from 2.96586 to 2.96532, saving model to /home/debjoy/towards_vortex/beginnersNNS/RNN/models_seqgen/weights-improvement-04-2.9653.hdf5
Epoch 5/20
1385/1385

In [16]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [17]:
import sys
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"  it rains
howl at the moon and go to sleep in the day
love for everybody 'til the drugs fade away
in "
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         